In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
# Table name
genre_bronzeTable = "bronze.genre"
movie_genre_bronzeTable = "bronze.movie_to_genre"

genre_silverTable = "silver.genre_dw"

In [0]:
df_genre = spark.read.table(genre_bronzeTable).select('id', 'name')
df_movie_genre = spark.read.table(movie_genre_bronzeTable)
df_genre.show()
df_movie_genre.show()

+---+-----------+
| id|       name|
+---+-----------+
|  1|     Action|
|  2|      Adult|
|  3|  Adventure|
|  4|  Animation|
|  5|  Biography|
|  6|     Comedy|
|  7|      Crime|
|  8|Documentary|
|  9|      Drama|
| 10|     Family|
| 11|    Fantasy|
| 12|  Film-Noir|
| 13|  Game-Show|
| 14|    History|
| 15|     Horror|
| 16|      Music|
| 17|    Musical|
| 18|    Mystery|
| 19|       News|
| 20| Reality-TV|
+---+-----------+
only showing top 20 rows

+-------+-------+
|movieId|genreId|
+-------+-------+
|      1|      6|
|      1|     11|
|      1|     21|
|      2|      7|
|      2|      9|
|      3|      9|
|      3|     15|
|      4|      7|
|      4|      9|
|      5|     15|
|      5|     18|
|      6|      3|
|      6|      9|
|      6|     14|
|      6|     27|
|      6|     28|
|      7|      6|
|      7|      9|
|      7|     21|
|      8|      9|
+-------+-------+
only showing top 20 rows



In [0]:
df_genre = df_genre.dropDuplicates(['id'])
df_movie_genre = df_movie_genre.dropDuplicates()


In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:

# Define the silver location

# Define the silver location
genre_silverLocation = "/mnt/datalake/silver/cleaned_genre_dw"
movie_genre_silverLocation = "/mnt/datalake/silver/cleaned_movie_genre"

# Table name
genre_silverTable = "silver.genre_dw"
movie_genre_silverTable = "silver.movie_genre"


df_genre.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(genre_silverLocation)
df_movie_genre.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(movie_genre_silverLocation)

In [0]:
# create the schema and table, if required
# spark.databricks.delta.formatCheck.enabled false

spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {genre_silverTable} USING delta LOCATION '{genre_silverLocation}'")
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {movie_genre_silverTable} USING delta LOCATION '{movie_genre_silverLocation}'")


DataFrame[]

In [0]:
# Maintenance for Delta table
genre_DataDelta = DeltaTable.forName(spark, genre_silverTable)

if genre_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	genre_DataDelta.optimize()
	genre_DataDelta.vacuum()


In [0]:
# Maintenance for Delta table
movie_genre_DataDelta = DeltaTable.forName(spark, movie_genre_silverTable)

if movie_genre_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	movie_genre_DataDelta.optimize()
	movie_genre_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.genre_dw;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-12-17T14:05:52Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(4283155382243377),1122-230829-ksqo7xkx,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-17T14:05:48Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(4283155382243377),1122-230829-ksqo7xkx,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-17T14:05:30Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(4283155382243377),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 28, numOutputBytes -> 1254)",null,Databricks-Runtime/13.3.x-scala2.12
